##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Game Design Brainstorming

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to aid in your game design process by fostering brainstorming sessions effectively.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Game_Design_Brainstorming.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies

Install Keras and KerasNLP.

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

## Core Concept

In [ ]:
# A super concise description of your game
pitch = "In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution."

## Load Model

In [ ]:
import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

import textwrap
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_7b_en")
gemma_lm.compile(sampler="top_k")


class GemmaChat:

    __START_TURN__ = "<start_of_turn>"
    __END_TURN__ = "<end_of_turn>"

    def __init__(self, model, system="", history=None):
        self.model = model
        self.system = system
        if not history:
            self.history = []
        else:
            self.history = history

    def format_message(self, message, prefix=""):
        return f"{self.__START_TURN__}{prefix}\n{message}{self.__END_TURN__}\n"

    def add_to_history(self, message, prefix=""):
        formated_message = self.format_message(message, prefix)
        self.history.append(formated_message)

    def get_full_prompt(self):
        prompt = "\n".join([self.system + "\n", *self.history])
        return prompt

    def send_message(self, message):
        self.add_to_history(message, "user")
        prompt = self.get_full_prompt()
        response = self.model.generate(prompt, max_length=4096)
        # print('--------------------response---------------------------')
        # print(response)
        # print('--------------------response---------------------------')
        result = response.replace(prompt, "")
        self.add_to_history(result, "model")
        return result

    def show_history(self):
        for h in self.history:
            print(h)


chat = GemmaChat(gemma_lm)

## Elaborate with the given core game concept

In [ ]:
result = chat.send_message(
    f"Elaborate about this game with the given core concept below.\n{pitch}"
)
to_markdown(result)

> **Core Concept:** Clockpunk fantasy meets historical intrigue in a meticulously crafted Joseon Dynasty setting.
> 
> **Possible Gameplay Elements:**
> - Unravel secrets of a lost empire using interactive historical documents.
> - Craft and upgrade your own mechanical gadgets from a diverse array of components.
> - Engage in political machinations and forge alliances with various factions.
> - Solve intricate puzzles and navigate perilous landscapes with the aid of your mechanical contraptions.
> 
> 
> **Additional Considerations:**
> - The game should offer multiple storylines and endings, reflecting different player choices.
> - Character customization and faction allegiances should play a significant role.
> - A rich lore and detailed worldbuilding should be established to immerse players in the unique setting.
> 
> **Target Audience:** Fans of historical fiction, clockpunk, puzzle-solving games, and strategy RPGs.
> 
> 
> **Possible Title:** The Cogborn Emperor
> 
> **Possible Setting:** Joseon Dynasty, reimagined with steam-powered technology and mechanical contraptions.
> 
> **Possible Tone:** Intriguing, mysterious, and strategic.
> 
> **Possible Gameplay Mechanics:**
> - **Gadget Crafting:** Gather resources and assemble various mechanical devices from scratch.
> - **Document Analysis:** Interpret historical records to uncover hidden secrets and clues.
> - **Political Intrigue:** Negotiate with different factions and sway them towards your cause.
> - **Puzzle-Solving:** Use your mechanical contraptions to solve environmental and narrative puzzles.
> 
> **Additional Notes:**
> - The game should feature a diverse cast of characters, each with their own unique motivations and abilities.
> - The soundtrack should enhance the atmosphere and setting.
> - The game should offer high replayability with multiple playthroughs and different outcomes.
> 
> ## Potential Story Arcs:
> 
> **1. The Forbidden Invention:** A clandestine inventor discovers a lost technology that threatens to unravel the established order.
> 
> **2. The Forgotten Emperor:** A deposed emperor returns from exile to reclaim his throne and restore his lost glory.
> 
> **3. The Shadow Conspiracy:** A shadowy cabal seeks to manipulate events from the shadows, leaving their motives unclear.
> 
> ## Potential Endings:
> 
> **1. The Peaceful Revolution:** The people rise up and overthrow the corrupt ruling class.
> 
> **2. The Imperial Restoration:** The current emperor maintains control, but agrees to reforms.
> 
> **3. The Mechanical Age:** The nation embraces technological advancement, transforming itself into a powerful empire.

## Main Characters

In [ ]:
result = chat.send_message("Design main characters.")
to_markdown(result)

> ## Main Character Concepts:
> 
> **1. Park Su-hyun:** A cunning strategist and inventor, haunted by the loss of his family in the political upheaval.
> 
> **2. Seo-yeon:** A skilled engineer with a passion for uncovering lost technology, driven by a thirst for knowledge.
> 
> **3. Han-joon:** A ruthless politician, manipulating events from the shadows for his own power.
> 
> **4. Jeong-ah:** A master spy, skilled in gathering information and uncovering secrets.
> 
> **5. Yi-seo:** A cunning strategist and former general, seeking redemption for past mistakes.

## Villain Characters

In [ ]:
result = chat.send_message("Design villain characters.")
to_markdown(result)

> ## Villain Concepts:
> 
> **1. The Whispering Shadow:** A mysterious entity manipulating events from the shadows, seeking to plunge the nation into eternal darkness.
> 
> **2. The Iron Chancellor:** A power-hungry politician who seeks to maintain his control at all costs.
> 
> **3. The Clocksmith:** A twisted inventor who has fused his consciousness with a mechanical clockwork, driven by a desire to reshape reality.
> 
> **4. The Aether Weaver:** A vengeful alchemist who seeks to consume the nation in an ethereal energy storm.
> 
> **5. The Serpent Guild:** A clandestine organization of assassins, driven by an insatiable thirst for power.

## Main Story

In [ ]:
result = chat.send_message(
    "Write the main story of this game with an introduction, development, turn, and conclusion."
)
to_markdown(result)

> ## The Cogborn Emperor: A Tale of Revolution and Steam
> 
> **Introduction:**
> 
> Within the reimagined Joseon Dynasty, a whisper of change hangs in the air. Steam-powered machines and intricate clockwork devices have become the hallmarks of a nation on the brink of transformation. But beneath the glittering advancements lie shadows, secrets, and a yearning for a forgotten past.
> 
> You are an individual of remarkable intellect and determination, bound by destiny to unravel the mysteries of a lost empire. In the heart of this technological revolution, you find yourself entangled in a web of political intrigue and ancient prophecies.
> 
> **Development:**
> 
> Your journey begins in a clandestine meeting, where you are approached by a shadowy figure who reveals a devastating truth: a powerful entity seeks to manipulate events from the shadows, threatening the balance of power in the nation. Armed with your wit, your mechanical ingenuity, and the loyalty of your trusted allies, you must uncover the secrets of the past to uncover the identity of the villain and their motives.
> 
> Through your travels, you encounter diverse factions, each with their own political agendas. You must navigate political negotiations, forge alliances, and make strategic decisions that will shape the fate of the nation.
> 
> **Turn:**
> 
> As you delve deeper into the mystery, you discover your own personal connection to the conspiracy. The lines between reality and illusion blur as the fate of the empire hangs in the balance. You must make difficult choices, sacrificing relationships and risking your own life for the greater good.
> 
> **Conclusion:**
> 
> Your ultimate challenge awaits in the heart of the conspiracy, where you must face the mastermind behind the shadows. With your skills and the power of your allies, you must uncover the villain's true identity reveal their motivations and bring them to justice.
> 
> The fate of the nation rests in your hands. Will you usher in a new era of peace and enlightenment, or will you sacrifice some ideals for the greater good? The choice is yours.

## Show the chat history

In [ ]:
chat.show_history()

<start_of_turn>user
Elaborate about this game with the given core concept below.
In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution.<end_of_turn>

<start_of_turn>model
**Core Concept:** Clockpunk fantasy meets historical intrigue in a meticulously crafted Joseon Dynasty setting.

**Possible Gameplay Elements:**
- Unravel secrets of a lost empire using interactive historical documents.
- Craft and upgrade your own mechanical gadgets from a diverse array of components.
- Engage in political machinations and forge alliances with various factions.
- Solve intricate puzzles and navigate perilous landscapes with the aid of your mechanical contraptions.


**Additional Considerations:**
- The game should offer multiple storylines and endings, reflecting different player choices.
- Character customi

## Clear History

In [ ]:
chat.history.clear()

## Expansion Idea

### Marketing Phrase

In [ ]:
chat.history.clear()
pitch = "A new steam-powered toothbrush"
result = chat.send_message(
    f"Generate a marketing phrase for the new product below.\n{pitch}"
)
to_markdown(result)

> * Cleans teeth with precision and efficiency
> * Uses natural steam power for a deeper clean
> * Gentle on teeth and gums
> 
> **Marketing Phrase Options:**
> 
> * **Steam to a Brighter Smile**
> * **The Power of Steam for Superior Oral Care**
> * **Experience a Clean Beyond Compare**
> * **Gentle Steam, Powerful Clean**
> * **Steam-Powered Precision for Optimal Oral Health**
> 
> 
> **Which marketing phrase would be most effective to highlight the unique benefits of the steam-powered toothbrush?**
> 
> I believe **"Steam-Powered Precision for Optimal Oral Health"** is the most effective marketing phrase because:
> 
> * **Clearly conveys the technology:** "Steam-Powered" instantly communicates the unique operating mechanism of the toothbrush.
> * **Highlights precision and efficiency:** "Precision" suggests careful cleaning, while "efficiency" implies a thorough clean in less time.
> * **Focuses on oral health:** The phrase emphasizes the toothbrush's ultimate goal of optimal oral health. 
> 
> **Additionally, this phrase:**
> 
> * Uses strong action-oriented language.
> * Appeals to both practical and aesthetic concerns.
> * Is concise and memorable.

### Florist Ideas

In [ ]:
chat.history.clear()
pitch = "Universe and shooting stars"
result = chat.send_message(
    f"Generate a florist idea inspired by the concept below, along with suggestions for suitable flowers.\n{pitch}"
)
to_markdown(result)

> 
> 
> **Florist Idea:**
> 
> **"Cosmic Embrace"**
> 
> **Concept:** A breathtaking arrangement inspired by the vastness of the cosmos, featuring ethereal flowers representing celestial bodies. 
> 
> 
> **Suitable Flowers:**
> 
> - **Cosmos:** Represent the delicate balance of energy in the universe.
> - **Stargazer lilies:** Symbolize reaching for the stars and boundless potential.
> - **Aster:** Known for their star-shaped blooms, representing shooting stars.
> - **Blue hydrangea:** Suggests the deep mysteries of the cosmos.
> - **Lavender:** Represents growth and transformation, echoing the evolution of the universe.
> 
> **Additional Elements:**
> 
> - Silver or mirrored glass vase to evoke the reflective surface of space.
> - Silver leaf foliage to create a shimmering, cosmic effect.
> - Dried flower petals resembling cosmic dust, adding a touch of etherealness.

### Food Recipe

In [ ]:
chat.history.clear()
pitch = "Cyberpunk Kraken"
result = chat.send_message(
    f"Generate a cooking recipe with the concept below.\n{pitch}"
)
to_markdown(result)

> The Kraken is a creature of the deep sea that is often depicted as a giant octopus. In this recipe, we will give the Kraken a cybernetic upgrade with the addition of some high-tech ingredients like LED lights and microchips. 
> 
> **Ingredients:**
> 
> - 2 pounds of fresh octopus tentacles
> - 1/2 cup of soy sauce
> - 1/4 cup of sake or rice wine
> - 1 tablespoon of cornstarch
> - 2 tablespoons of vegetable oil
> - 1/4 cup of LED lights
> - 1 microchip (optional)
> 
> **Instructions:**
> 
> 1. **Prepare the Kraken:** Cut the octopus tentacles into bite-sized pieces. In a bowl, combine the octopus pieces with the soy sauce, sake, cornstarch, and vegetable oil. Toss to coat.
> 
> 
> 2. **Cyberize the Kraken:** Arrange the octopus pieces on a baking sheet and sprinkle the LED lights over the top. Bake for 10-15 minutes or until the LED lights are glowing and the octopus is cooked through.
> 
> 
> 3. **Microchip enhancement (Optional):** If desired, insert a microchip into one of the octopus pieces before baking. This will give the Kraken a unique ability to glow in the dark.
> 
> 
> 4. **Serve and enjoy:** Serve the cyberpunk Kraken immediately with your favorite dipping sauce.
> 
> **Tips:**
> 
> - For a more realistic effect, serve the cyberpunk Kraken with a side of glowing seaweed or other marine plants.
> - If you're feeling adventurous, add some chopped chili peppers to the marinade for a spicy kick. 
> - Serve with a side of crusty bread to soak up the delicious sauce.